# HTR Kraken Cremma

Installation : 
- `pip install kraken`
- https://kraken.re/main/index.html

MarketPlace on shell:
- kraken list
- kraken show <name>
- kraken get <name>

In [1]:
import kraken

import cv2
from PIL import Image
import json
import matplotlib.pyplot as plt
from difflib import SequenceMatcher
import time

# Config

In [2]:
def Print(img, name = "img"):
    # Convert BGR to RGB and print it
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.title(name)
    plt.show()

def Image(name : str):
    img = cv2.imread(r'image/' + name)
    if img is None:
        print("Image Not Found")
    return img

def Score(original : str, transcrip : str):
    if transcrip == "":
        return -1
    else :
        return SequenceMatcher(None, original, transcrip).ratio()

In [8]:
def Timer(name:str, start_time:float):
    actual_time = time.time()
    print(f"{name} {actual_time - start_time:.3f} sec")
    return actual_time

# OCR

In [5]:
from kraken import binarization, pageseg

In [9]:
img_name = r'img_1.jpg'
#img = cv2.imread(r'image/' + img_name)
im = Image.open(r'image/' + img_name)
start_time = time.time()

bw_im = binarization.nlbin(im)
binarization_time = Timer("binarization.nlbin", start_time)

seg = pageseg.segment(bw_im)
pageseg_time = Timer("pageseg.segment", binarization_time)

binarization.nlbin 5.176 sec
pageseg.segment 48.616 sec


: 